# Read Dataset

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd 
A = pd.read_csv("C:/Users/Arshadali Shaikh/Downloads/training_set (2).csv")
A.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


# Missing Data treatment

In [3]:
def replacer(df):
    for i in df.columns:
        if(df[i].dtypes == "object"):
            x = df[i].mode()[0]
            df[i]=df[i].fillna(x)
        else:
            x = df[i].mean()
            df[i]=df[i].fillna(x)

In [4]:
replacer(A)  # treat the missing data.

In [5]:
A.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Define X and Y

In [6]:
Y = A[["Loan_Status"]]
X = A.drop(labels=["Loan_ID","Loan_Status"],axis=1)

# EDA
Ycat,Xcon --> ANOVA

Ycat,Xcat --> chisquare

# Divide data in con and cat

In [7]:
cat = []
con = []
for i in X.columns:
    if(X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [8]:
from PM8 import ANOVA,chisquare

for i in con:
    print("--------","Loan_Status vs",i,"-------")
    print(ANOVA(A,"Loan_Status",i))

-------- Loan_Status vs ApplicantIncome -------
0.903
-------- Loan_Status vs CoapplicantIncome -------
0.142
-------- Loan_Status vs LoanAmount -------
0.368
-------- Loan_Status vs Loan_Amount_Term -------
0.604
-------- Loan_Status vs Credit_History -------
0.0


In [9]:
for i in cat:
    print("--------","Loan_Status vs",i,"-------")
    print(chisquare(A,"Loan_Status",i))

-------- Loan_Status vs Gender -------
0.7391461310869638
-------- Loan_Status vs Married -------
0.029608580862582116
-------- Loan_Status vs Dependents -------
0.3688663081366005
-------- Loan_Status vs Education -------
0.04309962129357355
-------- Loan_Status vs Self_Employed -------
1.0
-------- Loan_Status vs Property_Area -------
0.0021360187811644915


# Outliers

In [11]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [13]:
X1 = pd.DataFrame(ss.fit_transform(X[con]),columns=con)

In [14]:
X1.head(2)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,0.072651,-0.554368,0.000000,0.279851,0.45164
1,0.000000,-0.038610,-0.219273,0.279851,0.45164


In [15]:
from PM8wd import outliers
OL = outliers(X1)

In [19]:
#OL

In [20]:
X = X.drop(index=OL,axis=0)
Y = Y.drop(index=OL,axis=0)  # dropped the outliers from the dataset

In [21]:
X.index = range(0,X.shape[0])
Y.index = range(0,Y.shape[0])  # Reindex the data 

# Preprocessing

In [22]:
from PM8wd import preprocessing
Xnew = preprocessing(X)

In [24]:
#Xnew

# Create a model using logistic Regression

In [27]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,pred_tr)
ts_acc = accuracy_score(ytest,pred_ts)
print(tr_acc,ts_acc)

0.8091106290672451 0.8362068965517241


In [30]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion="entropy",random_state=21,max_depth=2)
model = dtc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,pred_tr)
ts_acc = accuracy_score(ytest,pred_ts)
print(tr_acc,ts_acc)

0.8134490238611713 0.8362068965517241


# Tree model

In [31]:
X = X[["Credit_History","CoapplicantIncome","Property_Area","Education","Married"]]


In [32]:
Xnew = preprocessing(X)

In [33]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [34]:
Xnew.shape

(577, 9)

In [35]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion="entropy",random_state=21,max_depth=2)
model = dtc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,pred_tr)
ts_acc = accuracy_score(ytest,pred_ts)

In [36]:
tr_acc

0.8134490238611713

In [37]:
ts_acc

0.8362068965517241

# Using RandomForest Algorithm

In [38]:
from sklearn.ensemble import RandomForestClassifier
dtc = RandomForestClassifier(criterion="entropy",random_state=21,max_depth=5)
model = dtc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,pred_tr)
ts_acc = accuracy_score(ytest,pred_ts)

In [39]:
tr_acc

0.8199566160520607

In [40]:
ts_acc

0.8448275862068966

# Gridsearch CV

In [41]:
tuning_grid = {"max_depth":range(2,5,1),"criterion":["gini","entropy"],"n_estimators":range(20,30,1)}

In [42]:
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state=21)
cv = GridSearchCV(rfc,tuning_grid,scoring="accuracy",cv=4)  #Tuning hyperparameter to get best value. 

cvmodel = cv.fit(xtrain,ytrain)


In [43]:
cvmodel.best_params_

{'criterion': 'gini', 'max_depth': 2, 'n_estimators': 20}

In [45]:
rfc = RandomForestClassifier(criterion='gini', max_depth=2, n_estimators= 20,random_state=21)
model = rfc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,pred_tr)
ts_acc = accuracy_score(ytest,pred_ts)
print(tr_acc,ts_acc)

0.8112798264642083 0.8448275862068966


# Getting Data ready for predictions

In [49]:
B = pd.read_csv("C:/Users/Arshadali Shaikh/Downloads/testing_set (1).csv")

In [50]:
B

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural


In [51]:
B1 = B[X.columns].head()

# Missing Data treatment

In [52]:
B1.isna().sum()

Credit_History       1
CoapplicantIncome    0
Property_Area        0
Education            0
Married              0
dtype: int64

In [57]:
replacer(B)

In [58]:
cat = []
con = []
for i in B.columns:
    if(B[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [59]:
B1 = pd.DataFrame(ss.transform(B[con]),columns=con)
B2 = pd.get_dummies(B[cat])
Bnew = B1.join(B2)

In [60]:
ready_for_pred = Bnew[Xnew.columns] # data ready for prediction.

In [61]:
predicted_final = model.predict(ready_for_pred)

In [62]:
E = pd.DataFrame([B.Loan_ID,predicted_final]).T
E.columns = ["Loan_ID","Loan_Status"]

In [63]:
E

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
...,...,...
362,LP002971,Y
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y
